<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# LinkedIn - Accept all invitations and send first message
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Accept_all_invitations_and_send_first_message.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=LinkedIn+-+Accept+all+invitations+and+send+first+message:+Error+short+description">Bug report</a>

**Tags:** #linkedin #content #operations #automation #invitation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Last update:** 2023-05-29 (Created: 2022-04-05)

**Description:** This notebook helps you quickly and easily accept all LinkedIn invitations and send a personalized introductory message to each new connection.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries


In [ ]:
import naas
from naas_drivers import linkedin
import pandas as pd
import openai

### Setup variables
[Learn how to get your cookies on LinkedIn](https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75)
- `li_at`: Cookie used to authenticate Members and API clients.
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation.
- `cron`: Cron params for naas scheduler. More information: https://crontab.guru/
- `first_message`: First message to be sent
- `limit`: Number of invitations to be sent

[Get your OpenAI API key here](https://openai.com/docs/api-overview/)
- `openai_api_key`: This is your API key for OpenAI. It's used to authenticate your application's requests to the OpenAI API.
- `prompt`: This is a string that's used as a prompt for OpenAI's text generation API. It will be help you classify people that interacted with your post

In [ ]:
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_COOKIE_LI_AT"
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_COOKIE_JSESSIONID"
cron = "0 9 * * FRI"
first_message = "Hello [FIRSTNAME], glad to connect with you, can I help in any way?"
limit = 30

openai_api_key = naas.secret.get("OPENAI_API_KEY") or "YOUR_OPENAI_API_KEY"
prompt = f"""
I am building Naas, the Universal open source data plaform.
I have 2 ideal customer profile, one is a 'data producer' with basic knowledge of Python that could use our Notebook templates to create plugins. 
These plugions are then distributed data via our NaasAI Chat interface. 
The other one is a 'data consummer' that will enjoy using NaasAI Chat for its basic LLMs integration but also interested in having its own data available, hence work with the data producer. 
I will give you the [OCCUPATION] from a profile I get from LinkedIn, you will return stricly and only one of the following values inside the simple quotes based on the best match 'DataProducer', 'DataConsumer', 'NotICP' or 'Don't know' if you don't find a plausible match with the first 3 values.
Don't put the results into quotes.
"""

## Model

### Get invitations received

In [ ]:
df_invitation = linkedin.connect(li_at, JSESSIONID).invitation.get_received()
df_invitation

### Add ICP

In [ ]:
def predict_category(
    openai_api_key,
    prompt,
    summary
):
    # Return TBD if not openai key is set
    if not openai_api_key:
        return "TBD"
    
    # Connect to openai
    openai.api_key = openai_api_key
    
    if summary:
        prompt = prompt.replace("[OCCUPATION]", summary)
    else:
        return "NotICP"

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0,
        max_tokens=60
    )
    return response.choices[0].text.split("\n")[-1].strip()

df_icp = df_invitation.copy()
df_icp['ICP'] = df_icp.apply(lambda row: predict_category(openai_api_key, prompt, row["OCCUPATION"]), axis=1)
df_icp = df_icp[df_icp["ICP"] != "NotICP"][:limit].reset_index(drop=True)
print("People ICP:", len(df_icp))
df_icp.head(1)

## Output


### Accept pending invitations received from "Profile"

In [ ]:
def accept_new_contact(df):
    df_accept = pd.DataFrame()

    # Loop
    for index, row in df.iterrows():
        fullname = row.FULLNAME
        status = row.INVITATION_STATUS
        invitation_id = row.INVITATION_ID
        shared_secret = row.SHARED_SECRET
        if status == "PENDING":
            print(fullname)
            tmp_df = linkedin.connect(li_at, JSESSIONID).invitation.accept(
                invitation_id, shared_secret
            )
            df_accept = pd.concat([df_accept, tmp_df])
    return df_accept


df_accept = accept_new_contact(df_icp)
df_accept

### Send first message to contact

In [ ]:
def send_first_message(df):
    # Loop
    for index, row in df.iterrows():
        fullname = row.FULLNAME
        firstname = row.FIRSTNAME
        profile_id = row.PROFILE_ID
        print(fullname)
        first_message = first_message.replace("[FIRSTNAME]", firstname)
        linkedin.connect(li_at, JSESSIONID).message.send(first_message, profile_id)

send_first_message(df_accept)

### Add Scheduler

In [ ]:
# Schedule your notebook every hour
naas.scheduler.add(cron=cron)

# -> Uncomment the line below to remove your scheduler
# naas.scheduler.delete()